# Preparação do ambiente

## Bibliotecas

In [2]:
import pandas as pd
import re
import warnings

from datetime import datetime
from datetime import timedelta

from pathlib import Path

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'
# Suppress FutureWarning messages
warnings.simplefilter(action='ignore', category=FutureWarning)

## Constantes e funções auxiliares

In [4]:
DATA_COLETA = "12/07/2024"
ROOT_FOLDER = r"D:\OneDrive - ANATEL\Regulatron"

actual_collect_date = datetime.strptime(DATA_COLETA, "%d/%m/%Y").strftime("%Y%m%d")
root_path = Path(ROOT_FOLDER)

In [5]:
def format_issue(row):

    marketplace = row['marketplace']
    descricao_produto = row['descricao_produto']
    arquivo_pagina = row['arquivo_pagina']
    url = row['url']
    
    qual_codigo_sch_fornecido = row['qual_codigo_sch_fornecido']
    info_fabricante = row['info_fabricante']
    info_tipo_produto = row['info_tipo_produto']
    info_modelos_certificado = row['info_modelos_certificado']
    info_ean_homologados = row['info_ean_homologados']
    info_modelo_ean = row['info_modelo_ean']
    
    pdf_folder = 'https://anatel365.sharepoint.com/:b:/r/sites/Desenvolvimentodeappfiscalizaoe-commerce/Documentos%20Compartilhados/General/Resultados/screenshots'

    issue_mask = f'''h1. {descricao_produto}
    
"Aquivo PDF do anúncio coletado":{pdf_folder}/{arquivo_pagina} 

---
    
h3. Informações do certificado de homologação:
    
*_Número de Homologação_*: {qual_codigo_sch_fornecido}
    
*_Nome do Fabricante_*: {info_fabricante}
    
*_Tipo do Produto_*: {info_tipo_produto}
    
*_Modelos do Certificado_*: {info_modelos_certificado}

---

h3. Informações do EAN:
   
*_EAN homologados_*: 
* {info_ean_homologados}

*_Modelos do EAN_*: 
* {info_modelo_ean}'''

    issue_mask_amostra = f'''h1. {descricao_produto}
    
"Link do anúncio da amostra":{url} 

'''


    if marketplace in ['Shopee', 'Casas Bahia']:
        return issue_mask_amostra
    else:
        return issue_mask

# Carga e preparação dos dados

## Base EAN homologados

In [6]:
file_ean = root_path / 'datasets/sch/lista_celulares_homologados_ean.xlsx'
df_ean = pd.concat([df for df in pd.read_excel(file_ean, sheet_name=None,dtype='str').values()])
df_ean = df_ean.dropna()

columns_to_keep = ['Número de Homologação','Código EAN', 'Modelo', 'Nome Comercial']
df_ean = df_ean[columns_to_keep].drop_duplicates()

df_ean['Código EAN'] = df_ean['Código EAN'].apply(lambda x: x.strip())
df_ean['Número de Homologação'] = df_ean['Número de Homologação'].apply(lambda x: x.zfill(12))

df_ean = df_ean.sort_values(by=['Número de Homologação', 'Código EAN'])

columns_to_keep = ['Modelo', 'Nome Comercial']
# df_ean['Modelo Completo'] = df_ean[columns_to_keep].apply(lambda row: ' | '.join(row),axis=1)
df_ean['Modelo Completo'] = df_ean[columns_to_keep].apply(lambda row: ': '.join(row),axis=1)

columns_to_keep = ['Número de Homologação', 'Código EAN']
df_sch_ean = df_ean[columns_to_keep].drop_duplicates()
# df_sch_ean['Lista EAN'] = df_sch_ean[columns_to_keep].groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
df_sch_ean['Lista EAN'] = df_sch_ean[columns_to_keep].groupby('Número de Homologação').transform(lambda x : '\n* '.join(x))
df_sch_ean = df_sch_ean.drop_duplicates(subset='Número de Homologação')
df_sch_ean = df_sch_ean[['Número de Homologação', 'Lista EAN']]


columns_to_keep = ['Número de Homologação', 'Modelo Completo']
df_sch_ean_modelo = df_ean[columns_to_keep].drop_duplicates()
# df_sch_ean_modelo['Lista Modelo'] = df_sch_ean_modelo[columns_to_keep].groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
df_sch_ean_modelo['Lista Modelo'] = df_sch_ean_modelo[columns_to_keep].groupby('Número de Homologação').transform(lambda x : '\n* '.join(x))
df_sch_ean_modelo = df_sch_ean_modelo.drop_duplicates(subset='Número de Homologação')

columns_to_keep = ['Número de Homologação', 'Lista Modelo']
df_sch_ean = df_sch_ean.merge(df_sch_ean_modelo[columns_to_keep])
df_sch_ean = df_sch_ean.reset_index(drop=True)

df_sch_ean.columns = ['qual_codigo_sch_fornecido', 'info_ean_homologados', 'info_modelo_ean']

df_sch_ean.head()

,qual_codigo_sch_fornecido,info_ean_homologados,info_modelo_ean
0,000032309185,7908426307013\n* 7908426307020\n* 790842630703...,23021RAAEG: Redmi Note 12\n* 23028RA60L: Redmi...
1,000512201993,0194252145517\n* 0194252145524\n* 019425214585...,A2783: iPhone SE
2,000672100953,7892509117791\n* 7892509117807\n* 789250911781...,SM-A725M/DS: A72
3,001472314550,7908426304814\n* 7908426304821\n* 790842630587...,2201123G: POCO X5 PRO 5G\n* 22101320G: POCO X5...
4,001641811035,7898590571986,HT-705 G: HT-705 3G Kids\n* HT-705 G: HT-705G ...


## Base SCH

In [7]:
%%time

file_sch = root_path / 'datasets/sch/produtos_certificados.zip'
file_cel_5g = root_path / 'datasets/sch/celulares_5g_homologados.zip'
file_sch_consolidada = root_path / 'datasets/sch/base_sch_consolidada.parquet'

update_homologados = False

if update_homologados:
    
    dtype_sch = {'Número de Homologação': 'str'}
    dtype_cel_5g = {'NumeroHomologacao': 'str'}
    
    sch_columns_to_keep = ['Número de Homologação', 'Nome do Fabricante', 'Tipo do Produto', 'Modelo', 'Nome Comercial']
    cel_5g_columns_to_keep = ['NumeroHomologacao', 'Fabricante', 'TipodeProduto', 'Modelo', 'NomeComercial'] 
    
    df_sch = pd.read_csv(file_sch, sep=';',dtype=dtype_sch)
    df_sch = df_sch[sch_columns_to_keep]
    
    df_cel_5g = pd.read_csv(file_cel_5g, sep=';',dtype=dtype_cel_5g)
    df_cel_5g = df_cel_5g[cel_5g_columns_to_keep]
    df_cel_5g.columns = sch_columns_to_keep
    
    df_sch = pd.concat([df_sch, df_cel_5g])
    df_sch = df_sch.drop_duplicates()
    df_sch = df_sch.fillna('')
    
    df_tipo_produto = df_sch[['Número de Homologação','Tipo do Produto']].drop_duplicates()
    df_tipo_produto['Tipo Consolidado'] = df_tipo_produto.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_tipo_produto = df_tipo_produto[['Número de Homologação', 'Tipo Consolidado']].drop_duplicates()
    df_tipo_produto.columns = ['Número de Homologação', 'Tipo do Produto']
    
    df_modelo = df_sch[['Número de Homologação','Modelo']].drop_duplicates()
    df_modelo['Modelo Consolidado'] = df_modelo.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_modelo = df_modelo[['Número de Homologação', 'Modelo Consolidado']].drop_duplicates()
    df_modelo.columns = ['Número de Homologação', 'Modelo']
    
    df_nome_comercial = df_sch[['Número de Homologação','Nome Comercial']].drop_duplicates()
    df_nome_comercial['Nome Comercial Consolidado'] = df_nome_comercial.groupby('Número de Homologação').transform(lambda x : ' | '.join(x))
    df_nome_comercial = df_nome_comercial[['Número de Homologação', 'Nome Comercial Consolidado']].drop_duplicates()
    df_nome_comercial.columns = ['Número de Homologação', 'Nome Comercial']
    
    df_modelo_completo = df_modelo.merge(df_nome_comercial,how='left')
    df_modelo_completo['Modelos do Certificado'] = df_modelo_completo[['Modelo','Nome Comercial']].apply(lambda row: ' | '.join(row),axis=1)
    
    df_homologados = df_sch[['Número de Homologação','Nome do Fabricante']].drop_duplicates()
    df_homologados = df_homologados.merge(df_tipo_produto,how='left')
    df_homologados = df_homologados.merge(df_modelo_completo,how='left')
    
    columns_to_keep = ['Número de Homologação', 'Nome do Fabricante', 'Tipo do Produto', 'Modelos do Certificado']
    df_homologados = df_homologados[columns_to_keep]
    
    df_homologados.columns = ['qual_codigo_sch_fornecido', 'info_fabricante', 'info_tipo_produto', 'info_modelos_certificado']
    df_homologados.to_parquet(file_sch_consolidada)

else:
    df_homologados = pd.read_parquet(file_sch_consolidada)

df_homologados.head()

CPU times: total: 15.6 ms
Wall time: 103 ms


,qual_codigo_sch_fornecido,info_fabricante,info_tipo_produto,info_modelos_certificado
0,000370201543,Murandi Communications Ltd.,Equipamento de Radiocomunicação de Radiação Re...,SDM-TRIAX100 | SDM-TRIAX100
1,002070402205,K-best Technology Inc,Amplificador de Potência RF,BL2011N9 | BL2012N9 | BL2011N9 | BL2012N9
2,021050704223,Current Technologies International,Modem PLC,APA-2000-DB | APA-2000-DB
3,027890903257,Huawei Technologies Co. Ltd,Equipamento de Rede de Dados,Quidway S5328C-SI | Quidway S5328C-SI
4,000080201544,"PCTel, Inc.",Modem Analógico,PCT2304WT | PCT2304WT


## Resultados da raspagem dos marketplaces

In [8]:
map_columns = {
    'Item': 'item',
    'Texto da Busca' : 'texto_busca',
    'Arquivo da Página' : 'arquivo_pagina',
    'Data da Análise' : 'data_coleta',
    'Data da Coleta' : 'data_coleta',
    'Endereço eletrônico (URL)' : 'url',
    'Descrição do produto no anúncio' : 'descricao_produto',
    'O tipo do produto é de uso proibido no Brasil?' : 'produto_proibido',
    'O produto é passível de homologação na Anatel? (Sim ou Não)' : 'produto_passivel',
    'Tipo do Produto' : 'tipo_produto',
    'Fabricante' : 'fabricante',
    'Modelo do Produto' : 'modelo',
    'Valor do Produto (R$)' : 'valor',
    'Número de Unidades à Venda' : 'unidades_venda',
    'Existe o campo código de homologação no anúncio? (Sim ou Não)' : 'existe_campo_codigo_sch',
    'O código de homologação foi fornecido? (Sim ou Não)' : 'codigo_sch_foi_fornecido',
    'O produto é homologado? (Sim, Não e N.A.)' : 'produto_homologado',
    'Validação do código de homologação - O código de homologação fornecido é o do produto anunciado? (Sim, Não, N.A.)' : 'codigo_sch_corresponde_produto',
    'O código EAN foi fornecido? (Sim ou Não) - Apenas para Smartphones.' : 'codigo_ean_foi_fornecido',
    'O código EAN fornecido corresponde ao produto? (Sim ou Não) -  Apenas para Smartphones.' : 'codigo_ean_corresponde_produto',
    'Observação 1 - Qual Código de Homologação fornecido no anúncio?' : 'qual_codigo_sch_fornecido',
    'Marketplace' : 'marketplace',
    'Página': 'pagina',
    'Arquivo': 'arquivo_pagina',
    'Data' : 'data_coleta',
    'URL' : 'url',
    'Título': 'descricao_produto',
    'Categoria': 'tipo_produto',
    'Modelo': 'modelo',
    'Preço': 'valor',
    'Unidades à Venda': 'unidades_venda',
    'Existe campo código SCH?' : 'existe_campo_codigo_sch',
    'Código SCH foi fornecido?' : 'codigo_sch_foi_fornecido',
    'Código de Homologação': 'qual_codigo_sch_fornecido',
    'Código SCH é o do produto?': 'codigo_sch_corresponde_produto',
    'O produto é homologado?': 'produto_homologado',
    'O código EAN foi fornecido?': 'codigo_ean_foi_fornecido',
    'Código EAN é o do produto?': 'codigo_ean_corresponde_produto',
    'Código EAN-GTIN': 'qual_codigo_ean_fornecido'
}

In [9]:
results_folder = root_path
result_files = [file for file in results_folder.rglob('*.xlsx') if actual_collect_date in file.name]

map_marketplace = {
    'amazon': 'Amazon', 
    'ml': 'Mercado Livre',
    'carrefour': 'Carrefour', 
    'magalu': 'Magazine Luiza',
    'shopee': 'Shopee',
    'americanas': 'Americanas',
    'casasbahia': 'Casas Bahia'
}

map_tipo_produto = {
    'Celulares e Smartphones': 'Celulares e Smartphones',
    'smartphone': 'Celulares e Smartphones',
    'Smartphones': 'Celulares e Smartphones'
}

scrape_results = []

for file in result_files:
    
    file_name = file.name
    marketplace_code = file_name.split('_')[0]
    marketplace = map_marketplace[marketplace_code]

    df = pd.read_excel(file,dtype='str')
    # remove os caracteres R$ 
    df['Preço'] = df['Preço'].apply(lambda x: str(x).replace(r'R$','').strip())
    # remove '.' separador de milhares e troca separador de decimais de ',' para '.'
    df['Preço'] = df['Preço'].apply(lambda x: x.replace('.','').replace(',','.') if ',' in x else x)
    
    df['Marketplace'] = marketplace
    
    scrape_results.append(df)

df_produtos = pd.concat(scrape_results)

df_produtos = df_produtos.reset_index(drop=True)
df_produtos.columns = df_produtos.columns.map(map_columns)
df_produtos['tipo_produto'] = df_produtos['tipo_produto'].map(map_tipo_produto)

columns_to_keep = ['tipo_produto', 'texto_busca', 'data_coleta', 'marketplace',
                   'arquivo_pagina', 'url', 'descricao_produto',
                   'fabricante', 'modelo', 'valor', 
                   'existe_campo_codigo_sch', 'codigo_sch_foi_fornecido', 'qual_codigo_sch_fornecido', 'codigo_sch_corresponde_produto',
                   'codigo_ean_foi_fornecido', 'qual_codigo_ean_fornecido', 'codigo_ean_corresponde_produto']
df_produtos = df_produtos[columns_to_keep]

df_produtos = df_produtos.merge(df_homologados,on='qual_codigo_sch_fornecido',how='left')
df_produtos = df_produtos.merge(df_sch_ean,on='qual_codigo_sch_fornecido',how='left')
df_produtos['info_ean_homologados'].fillna('EAN indisponível',inplace=True)
df_produtos['info_modelo_ean'].fillna('EAN indisponível',inplace=True)
df_produtos = df_produtos.fillna('')

valor_pattern = '[0-9]+\.?(?:[0-9]{2})?'
df_produtos['valor'] = df_produtos['valor'].apply(lambda x: x if re.match(valor_pattern,x) else -1).astype(float)

# strip pipe caracter from the end of string
for col in ['info_tipo_produto', 'info_modelos_certificado', 'info_ean_homologados']:
    df_produtos[col] = df_produtos[col].apply(lambda x: x[:-3] if x[-3:]==' | ' else x)

df_produtos

<>:57: SyntaxWarning: invalid escape sequence '\.'
<>:57: SyntaxWarning: invalid escape sequence '\.'
C:\Users\rsilva\AppData\Local\Temp\ipykernel_25792\1916860417.py:57: SyntaxWarning: invalid escape sequence '\.'
  valor_pattern = '[0-9]+\.?(?:[0-9]{2})?'


,tipo_produto,texto_busca,data_coleta,marketplace,arquivo_pagina,url,descricao_produto,fabricante,modelo,valor,...,qual_codigo_sch_fornecido,codigo_sch_corresponde_produto,codigo_ean_foi_fornecido,qual_codigo_ean_fornecido,codigo_ean_corresponde_produto,info_fabricante,info_tipo_produto,info_modelos_certificado,info_ean_homologados,info_modelo_ean
0,Celulares e Smartphones,smartphone,12/07/2024,Amazon,amazon_20240712_B0BDPP5JWB.pdf,https://www.amazon.com.br/Samsung-Galaxy-dobrá...,Samsung Galaxy Z Fold4 256GB 5G Wi-Fi Tela dob...,Samsung,SM-F936BZAJZTO,9723.17,...,086722200953,,Não,,,Samsung Electronics Co Ltd.,Equipamento de Radiocomunicação de Radiação Re...,SM-F936B/DS,7892509125949\n* 7892509125956\n* 789250912596...,SM-F936B/DS: Z Fold4 5G (256GB)\n* SM-F936B/DS...
1,Celulares e Smartphones,smartphone,12/07/2024,Amazon,amazon_20240712_64.pdf,https://www.amazon.com.br/Apple-iPhone-15-256-...,Apple iPhone 15 (256 GB) — Amarelo,Apple,,5599.99,...,127572301993,,Não,,,Apple Inc.,Transceptor de Radiação Restrita | Transceptor...,A3090 | iPhone 15,0195949035913\n* 0195949035920\n* 019594903609...,A3090: iPhone 15
2,Celulares e Smartphones,smartphone,12/07/2024,Amazon,amazon_20240712_B0CMJJZF2G.pdf,https://www.amazon.com.br/Smartphone-Xiaomi-Di...,Smartphone Xiaomi POCO X6 Pro 5G 8GB+256GB NFC...,Xiaomi,2311DRK48G | POCO X6 Pro 5G | 2311DRK48G,1960.00,...,217522314550,,Não,,,"Xiaomi Communications Co., Ltd.",Transceptor de Radiação Restrita | Sistemas de...,2311DRK48G | POCO X6 PRO 5G,7908426309147\n* 7908426309154\n* 7908426309437,2311DRK48G: POCO X6 PRO 5G
3,Celulares e Smartphones,smartphone,12/07/2024,Amazon,amazon_20240712_B0BDBKX5JW.pdf,https://www.amazon.com.br/Smartphone-Motorola-...,Smartphone Motorola Moto E22 4G 32GB 2GB RAM G...,Motorola,XT2239-16 | Motorola E22,979.90,...,126752200330,,Não,,,Motorola Mobility LLC,Transceptor de Radiação Restrita | Telefone Mó...,XT2239-10 | XT2239-16 | Moto e22,7892597352371\n* 7892597352395\n* 789259735241...,XT2239-16: Motorola moto e22\n* XT2239-10: Mot...
4,Celulares e Smartphones,smartphone,12/07/2024,Amazon,amazon_20240712_B0CD2Y7DK5.pdf,https://www.amazon.com.br/Smartphone-Motorola-...,Smartphone Motorola Razr 40 5G 256GB 8GB RAM L...,Motorola,XT2323-1,4299.00,...,062482300330,,Não,,,Motorola Mobility LLC,Transceptor de Radiação Restrita | Sistemas de...,XT2323-1,7892597353217\n* 7892597353224\n* 7892597353231,XT2323-1: Motorola Razr 40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,Celulares e Smartphones,smartphone,11/07/2024,Shopee,,https://shopee.com.br/Samsung-Galaxy-m34-5g-12...,Samsung Galaxy m34 5g 128/6gb prata anatel,,,1199.00,...,,,,,,,,,EAN indisponível,EAN indisponível
429,Celulares e Smartphones,smartphone,11/07/2024,Shopee,,https://shopee.com.br/Celular-Xiaomi-Redmi-13c...,Celular Xiaomi Redmi 13c NFC Dual Sim Tela 6.6...,,,889.99,...,,,,,,,,,EAN indisponível,EAN indisponível
430,Celulares e Smartphones,smartphone,11/07/2024,Shopee,,https://shopee.com.br/Smartphone-Xiaomi-Redmi-...,Smartphone Xiaomi Redmi Note 13 5G Global 256G...,,,1489.99,...,,,,,,,,,EAN indisponível,EAN indisponível
431,Celulares e Smartphones,smartphone,11/07/2024,Shopee,,https://shopee.com.br/Xiaomi-Redmi-12C-Celular...,Xiaomi Redmi 12C Celular Original 128GB 4GB RA...,,,684.00,...,,,,,,,,,EAN indisponível,EAN indisponível


# Análise

In [10]:
map_fiscais_marketplace_even = {
    "Amazon": {"Atribuído para": "werther", "Unidade de Lotação": "GR03"},
    "Mercado Livre": {"Atribuído para": "costalo", "Unidade de Lotação": "UO021"},
    "Carrefour": {"Atribuído para": "marcosrg", "Unidade de Lotação": "UO072"},
    "Magazine Luiza": {"Atribuído para": "caldeira", "Unidade de Lotação": "GR04"},
    "Shopee": {"Atribuído para": "lribeiro", "Unidade de Lotação": "GR02"},
    "Americanas": {"Atribuído para": "humbertovinagre", "Unidade de Lotação": "GR01"},
    "Casas Bahia": {"Atribuído para": "leandrorb", "Unidade de Lotação": "GR07"},
}

map_fiscais_marketplace_odd = {
    "Amazon": {"Atribuído para": "yoshida", "Unidade de Lotação": "GR03"},
    "Mercado Livre": {"Atribuído para": "josias", "Unidade de Lotação": "UO021"},
    "Carrefour": {"Atribuído para": "denerreis", "Unidade de Lotação": "UO072"},
    "Magazine Luiza": {"Atribuído para": "marcelo", "Unidade de Lotação": "GR04"},
    "Shopee": {"Atribuído para": "danilo", "Unidade de Lotação": "GR02"},
    "Americanas": {"Atribuído para": "beraldi", "Unidade de Lotação": "GR01"},
    "Casas Bahia": {"Atribuído para": "parazi", "Unidade de Lotação": "FISF"},
}

data_inicio = DATA_COLETA
data_inicio = datetime.strptime(data_inicio, "%d/%m/%Y")
data_fim = data_inicio + timedelta(days=7)
data_fim = data_fim.strftime("%d/%m/%Y")

df_fiscaliza = df_produtos.copy()

# título da última tarefa no Fiscaliza
# as novas terão numeração sequencial a partir desta
fiscaliza_last_issue = 0
format_title = (
    lambda i: f"INSP_PACP_{actual_collect_date}_{str(i+1+fiscaliza_last_issue).zfill(6)}"
)

df_fiscaliza["Projeto"] = "Rotulagem de Dados"

df_fiscaliza["Título"] = df_fiscaliza.index.map(format_title)
df_fiscaliza["Descrição"] = df_fiscaliza.apply(lambda row: format_issue(row), axis=1)
df_fiscaliza["Tipo"] = "Fiscalização de Marketplace"
df_fiscaliza["Situação"] = "Responder pesquisa"
df_fiscaliza["Data de início"] = data_inicio
df_fiscaliza["Data limite"] = data_fim

# fiscais e unidade de lotação
# df_fiscaliza = df_fiscaliza.join(pd.DataFrame(df_fiscaliza['marketplace'].map(map_fiscais_marketplace).to_list()))
idx = df_fiscaliza.index.to_list()
even_idx = idx[0::2]
odd_idx = idx[1::2]
df_fiscais = pd.DataFrame(df_fiscaliza["marketplace"])
df_fiscais["fiscal_even"] = df_fiscais.iloc[even_idx]["marketplace"].map(
    map_fiscais_marketplace_even
)
df_fiscais["fiscal_odd"] = df_fiscais.iloc[odd_idx]["marketplace"].map(
    map_fiscais_marketplace_odd
)
df_fiscais["fiscal"] = df_fiscais[["fiscal_even", "fiscal_odd"]].apply(
    lambda row: row["fiscal_even"] if pd.isna(row["fiscal_odd"]) else row["fiscal_odd"],
    axis=1,
)
df_fiscais = pd.DataFrame(df_fiscais["fiscal"].to_list())
df_fiscaliza = df_fiscaliza.join(df_fiscais)

columns_to_issues = [
    "url",
    "Projeto",
    "Tipo",
    "Situação",
    "Título",
    "Descrição",
    "Data de início",
    "Data limite",
    "Atribuído para",
    "Unidade de Lotação",
    "tipo_produto",
    "texto_busca",
    "data_coleta",
    "marketplace",
    "fabricante",
    "modelo",
    "valor",
    "existe_campo_codigo_sch",
    "codigo_sch_foi_fornecido",
    "qual_codigo_sch_fornecido",
    "codigo_sch_corresponde_produto",
    "codigo_ean_foi_fornecido",
    "qual_codigo_ean_fornecido",
    "codigo_ean_corresponde_produto",
]
df_fiscaliza = df_fiscaliza[columns_to_issues]

columns_to_rename = [
    "url",
    "Projeto",
    "Tipo",
    "Situação",
    "Título",
    "Descrição",
    "Data de início",
    "Data limite",
    "Atribuído para",
    "Unidade de Lotação",
    "Qual o tipo do produto? ",
    "Texto da busca",
    "Qual foi a data da coleta?",
    "Marketplace",
    "Fabricante",
    "Modelo do Produto",
    "Valor",
    "Existe campo código SCH?",
    "Código SCH foi fornecido?",
    "Qual código SCH fornecido?",
    "Código SCH é o do produto?",
    "Código EAN foi fornecido?",
    "Qual código EAN fornecido?",
    "Código EAN é o do produto?",
]
df_fiscaliza.columns = columns_to_rename


df_fiscaliza["Qual foi a data da coleta?"] = datetime.today().strftime("%Y%m%d") #TODO: verificar bug na formatação de data

for collect_date in df_fiscaliza["Qual foi a data da coleta?"].unique():
    # issues_file_date = datetime.strptime(collect_date, "%Y-%m-%d").strftime("%Y%m%d")
    issues_file_date = pd.to_datetime(collect_date).strftime("%Y%m%d")
    issues_output_folder = root_path / "issues"
    issues_file_name = issues_output_folder / f"issues_{issues_file_date}.csv"
    if not issues_file_name.exists():
        df_fiscaliza[df_fiscaliza["Qual foi a data da coleta?"] == collect_date].to_csv(
            issues_file_name, index=False, sep=";", quotechar="'"
        )
    else:
        print(f"Issues file for collect date {collect_date} already exists. Not saved.")

df_fiscaliza

ValueError: time data '12/07/2024' does not match format '%Y-%m-%d'